In [1]:
# Read global variable OPENAI_KEY with your API key
import os
import openai
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import os
from langchain_openai  import ChatOpenAI
import pandas as pd

# OpenAI Key

In [2]:
os.environ['OPENAI_API_KEY'] = "your_openai_api_key"

# Examples

In [3]:
import json
with open("../data/few-shot-examples.json") as f:
    examples = json.load(f)

examples

[{'input': 'Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try

# Assemble into few-shot prompt template

In [4]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign

# Assemble final prompt with model

In [5]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant who enhances the user experience on news search engine results from pages. Specifically, you should identify the full headline and the key points of news articles. Each key point has a title and the main idea. Limited your answer to providing the title and the main key points without additional information, descriptions, or explanations."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# Idenitfy title and key points from content

In [6]:
df = pd.read_csv("../data/bbc-news-data(new).csv")
df.head()

,category,filename,label,title,content
0,business,001.txt,1,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarner...
1,business,002.txt,0,Dollar gains on Greenspan speech,The dollar has hit its highest level against t...
2,business,003.txt,0,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuko...
3,business,004.txt,1,High fuel prices hit BA's profits,British Airways has blamed high fuel prices fo...
4,business,005.txt,0,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domec...


In [7]:
title = df["title"].iloc[4]
content = df["content"].iloc[4]
question = "What are the title and key points from the following news article?"
input = f"{question}\n\n{title}\n\n{content}"

In [8]:
chain = final_prompt | ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, )

response = chain.invoke({"input": input})
print(response.content)

Title: Pernod takeover talk lifts Domecq

Key points:
1. Speculation that France's Pernod Ricard is considering a takeover bid for UK drinks and food firm Allied Domecq has caused a rise in Allied Domecq's shares.
2. Pernod Ricard, seeking acquisitions, has not confirmed the reports but has a history of significant purchases, including a third of US giant Seagram in 2000.
3. Pernod's market value is about 9% smaller than Allied Domecq, with Pernod owning brands like Chivas Regal, Havana Club, and Jacob's Creek, while Allied Domecq owns Malibu, Courvoisier, Stolichnaya, and Ballantine's, as well as snack food chains Dunkin' Donuts and Baskin-Robbins.
4. Both companies have addressed issues within their portfolios, with Pernod reducing debt from the Seagram purchase and Allied improving the performance of its fast-food chains.
